In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 6it [00:00, 26.92it/s, center_shift=0.000007, iteration=6, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Loading dataset from cache to device cuda
Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.3.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [39]:
decomp.lattice_scores[0].shape

(50, 50)

In [35]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 0.5):
    r = [(layer < related_cutoff) * layer for layer in lattice]
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 0.5):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice[-1] = cutoff_lattice(decomp.lattice_scores, related_cutoff=0.5)

utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# TODO:
""""
We need to combine the 
"""

import numpy as np

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
score_path = [25, 28, 20, 20, 20]
"""
Paths which seem to have meaning

"""

to_score = [d[:100] for d in ds][:300]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([1, 1, 1, 1, 1])
)

DS (5, 8250, 512) 8250
(8250, 5, 512) [25, 28, 20, 20, 20]


In [ ]:
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!

# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=[[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 100))]] for tok in s] for s in scores_reord],
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)

In [ ]:
scores_per_token_set, scores_per_token_set[top_args]

(array([1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04433304, 1.04433304,
        1.04433304, 1.04433304, 1.04433304, 1.04

In [ ]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_299492/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
